In [16]:
import pandas as pd

import random
import re
import pickle
import nltk

In [17]:
dataset = pd.read_csv("reviews.csv",sep="\t")
dataset.head(3)

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
0,0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,211.200.63.206
1,1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,192.17.224.99
2,2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,198.9.2.161


In [18]:
### WORKING ON TWITTER DATA FOR SENTIMENTAL ANALYSIS

with open("Pickle Files/classifier.pickle","rb") as f:
    clf = pickle.load(f)

with open("Pickle Files/TfidfModel.pickle","rb") as f:
    tfidf = pickle.load(f)

def getSentiment(text):

    # PREPROCESSING THE DATASET
    text = str(text)
    text = text.lower()
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain't","am not",text)
    text = re.sub(r"wouldn't","would not",text)
    text = re.sub(r"shouldn't","should not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"couldn't","could not",text)
    text = re.sub(r"won't","will not",text)
    
    text = re.sub(r"\W"," ",text)
    text = re.sub(r"\d"," ",text)
    text = re.sub(r"\s+[a-z]\s+"," ",text)
    text = re.sub(r"^[a-z]\s+"," ",text)    
    text = re.sub(r"\s+[a-z]$"," ",text)    
    text = re.sub(r"\s+"," ",text)    
    
    sent = clf.predict(tfidf.transform([text]).toarray())
    
    return sent[0]
            

## 1. different sentiment in review headline and review body

In [22]:
blocked_reviews = []
for i in range(len(dataset)):
        if( getSentiment( dataset["review_headline"][i] ) != getSentiment( dataset["review_body"][i] ) ):
            blocked_reviews.append(dataset["review_id"][i])

In [23]:
blocked_reviews

['R31LOQ8JGLPRLK', 'RY8DDL22YG4R5', 'R3RRXU2R23NMQ9']

## 2. Users which are posting either all positive or negative reviews on different products of same brand


In [24]:
dataset = pd.read_csv("sentiment-for-a-company-product-by-same-customer.csv")

In [25]:
# users which are posting either all positive or negative reviews on different products of same brand

customers = dataset.groupby("customer_id")
customer_list = dataset["customer_id"].unique()

blocked_users = []
size = len(customer_list.tolist())
index = 0
for i in range(size):
    brand_df = customers.get_group(customer_list[i])    
    brands = brand_df.groupby("product_parent")
    brands_list = brand_df["product_parent"].unique()
    
    no_of_brands = len(brands_list.tolist())
    for j in range(no_of_brands):
        
        product_df = brands.get_group(brands_list[j])
        no_of_products = len(product_df["product_id"])
        
        sentiment = getSentiment(product_df["review_body"][index])
                
        isSameSentiment = True
        if no_of_products==2:
            break
        for k in range(no_of_products):
            text = str(product_df["review_body"][index+k])
            if getSentiment(text)!=sentiment :
                isSameSentiment = False
                break;
        if(isSameSentiment):
            blocked_users.append(customer_list[i])
            break
    if(isSameSentiment):
        break;
    index += k

In [26]:
blocked_users

[20422322]

In [ ]:
'''
dataset.set_index("customer_id",inplace=True)
for customer in blocked_users:
    dataset.drop(customer,inplace=True)
dataset.reset_index()
'''

## 3.Reviews from same IP either all positive or negative reviews on different products of same brand

In [27]:
dataset = pd.read_csv("sentiment-for-a-company-product-by-same-IP.csv")

In [12]:
ip = dataset.groupby("IP Address")
ip_list = dataset["IP Address"].unique()

# Reviews from same IP either all positive or negative reviews on different products of same brand

blocked_users = []
size = len(ip_list.tolist())
index = 0
for i in range(size):
    brand_df = ip.get_group(ip_list[i])
    
    brands = brand_df.groupby("product_parent")
    brands_list = brand_df["product_parent"].unique()
    
    no_of_brands = len(brands_list.tolist())
    for j in range(no_of_brands):
        
        product_df = brands.get_group(brands_list[j])
        no_of_products = len(product_df["product_id"])
        
        sentiment = getSentiment(product_df["review_body"][index])
                
        isSameSentiment = True
        if no_of_products<=2:
            break
        for k in range(no_of_products):
            text = str(product_df["review_body"][index+k])
            if getSentiment(text)!=sentiment :
                isSameSentiment = False
                break;
        if(isSameSentiment):
            blocked_users.append(ip_list[i])
            break
    if(isSameSentiment):
        break;
    index +=k

In [51]:
blocked_users

['193.93.167.87']

In [53]:
'''
dataset.set_index("IP Address",inplace=True)
for ip in blocked_users:
    dataset.drop(ip,inplace=True)
dataset.reset_index()
'''

'\ndataset.set_index("IP Address",inplace=True)\nfor ip in blocked_users:\n    dataset.drop(ip,inplace=True)\ndataset.reset_index()\n'

## 4. User posting (>3) reviews on the same day with all the reviews are either positive or negative.

In [5]:
dataset.sort_values("customer_id",inplace=True)

In [8]:
dataset.head(1)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
24496,US,10071,R1XALU24Y932WC,B0048DLA90,864204912,NEW Click Wheel Flex Cable For Apple iPod Nano...,Mobile_Electronics,5,0.0,0.0,N,Y,Five Stars,Just what I needed. Thanks.,2014-07-31,217.163.92.95


In [33]:
dataset = pd.read_csv("same-day-multiple-reviews with same sentiment.csv")

In [34]:
customer_group = dataset.groupby("customer_id")
customer_group_list = dataset["customer_id"].unique().tolist()
index=0

for i in range(len(customer_group_list)):
    customer_reviews = customer_group.get_group( customer_group_list[i] )
    
    dates_list = customer_reviews["review_date"].unique().tolist()
    print(dates_list)
    reviews_by_date = customer_reviews.groupby("review_date");
    
    for j in range(len(dates_list)):
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        df = reviews_by_date.get_group(dates_list[j])
        
        for k in range(len(df)):
            text = df["review_body"][index+k]
            if(getSentiment(text) == 0):
                reviews_by_date_for_neg.append(df["review_id"][index+k])
            else:
                reviews_by_date_for_pos.append(df["review_id"][index+k])
        print(len(reviews_by_date_for_pos) , reviews_by_date_for_pos)
        print(len(reviews_by_date_for_neg) , reviews_by_date_for_neg)
        # count, review id of reviews with same sentiment
        index += k+1
        

['2015-08-31']
3 ['R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R2Y0MM9YE6OP3P']
1 ['RRB9C05HDOD4O']
['2015-09-31']
4 ['R26I2RI1GFV8QG', 'RY8DDL22YG4R5', 'R2AT2426ZHFUHH', 'R3RRXU2R23NMQ9']
0 []


dict_keys(['2015-09-31'])

## 5. Reviews(>3) from same IP on the same day with all the reviews are either positive or negative.

In [41]:
dataset = pd.read_csv("same-day-multiple-reviews with same sentiment from same IP.csv")

In [11]:
ip_group = dataset.groupby("IP Address")
ip_list = dataset["IP Address"].unique().tolist()
index=0

for i in range(1000):
    reviews = ip_group.get_group( ip_list[i] )
    
    dates_list = reviews["review_date"].unique().tolist()
    print(dates_list)
    reviews_by_date = reviews.groupby("review_date");
    
    for j in range(len(dates_list)):
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        df = reviews_by_date.get_group(dates_list[j])
        
        for k in range(len(df)):
            text = df["review_body"][index+k]
            if(getSentiment(text) == 0):
                reviews_by_date_for_neg.append(df["review_id"][index+k])
            else:
                reviews_by_date_for_pos.append(df["review_id"][index+k])
        print(len(reviews_by_date_for_pos) , reviews_by_date_for_pos)
        print(len(reviews_by_date_for_neg) , reviews_by_date_for_neg)
        # count, review id of reviews with same sentiment
        index += k+1
        

['2015-08-31']
1 ['R8MEA6IGAHO0B']
0 []
['2015-08-31']
1 ['R31LOQ8JGLPRLK']
0 []
['2015-08-31']
1 ['R2Y0MM9YE6OP3P']
0 []
['2015-08-31']
0 []
1 ['RRB9C05HDOD4O']
['2015-08-31']
1 ['R26I2RI1GFV8QG']
0 []
['2015-08-31']
1 ['RY8DDL22YG4R5']
0 []
['2015-08-31']
1 ['R2AT2426ZHFUHH']
0 []
['2015-08-31']
1 ['R3RRXU2R23NMQ9']
0 []
['2015-08-31']
1 ['R250PR8VJUZ62F']
0 []
['2015-08-31']
1 ['RBEMQ29WJBHYG']
0 []
['2015-08-31']
1 ['R19VVIUT4BZCMT']
0 []
['2015-08-31']
0 []
1 ['R1DT8JJUQHUKUL']
['2015-08-31']
1 ['RLIL6S9OGM3YA']
0 []
['2015-08-31']
1 ['R34EZZ68VYPHO0']
0 []
['2015-08-31']
0 []
1 ['R3TZNSOXS13SIG']
['2015-08-31']
1 ['R12UBZT87UX3AP']
0 []
['2015-08-31']
0 []
1 ['R5KCRE927NYMU']
['2015-08-31']
1 ['RT12RQFU0V2AX']
0 []
['2015-08-31']
0 []
1 ['R2K0YQPV5W2SJ1']
['2015-08-31']
1 ['R104MY31OKNGXH']
0 []
['2015-08-31']
1 ['R3EQYRGIIJQBHT']
0 []
['2015-08-31']
1 ['R32MW6D6CXKCFW']
0 []
['2015-08-31']
1 ['RFO6FJSAA391Z']
0 []
['2015-08-31']
1 ['R1IG9KI4EWNE4W']
0 []
['2015-08-31']
1 ['R3IF1

0 []
1 ['R36E4H31GBXB37']
['2015-08-27']
1 ['R30TJ8POYNVCRE']
0 []
['2015-08-27']
0 []
1 ['R3QHA5X5VBDVAP']
['2015-08-27']
0 []
1 ['RJQNG7DPY6EEI']
['2015-08-27']
0 []
1 ['R1YRP4MPCUD8WB']
['2015-08-27']
0 []
1 ['R39LB21NJRO8VT']
['2015-08-27']
1 ['R22CLKR9Y3035O']
0 []
['2015-08-27']
1 ['R1KTAIJ9NN4VC9']
0 []
['2015-08-27']
0 []
1 ['R32LCL8SAK1I1']
['2015-08-27']
1 ['R3SW1K55Z20RYL']
0 []
['2015-08-27']
1 ['R2NWNV371VKOM3']
0 []
['2015-08-27']
1 ['R2G9U63HXYJAJ8']
0 []
['2015-08-27']
0 []
1 ['R1CM91W9HYDP5V']
['2015-08-27']
0 []
1 ['R2SWL4208CK2SI']
['2015-08-27']
1 ['R2I61D84SVYIW7']
0 []
['2015-08-27']
1 ['R1MA4THSYOMMH1']
0 []
['2015-08-27']
1 ['RP0AQMVPR45U2']
0 []
['2015-08-27']
1 ['R85I6JQCCXIR9']
0 []
['2015-08-27']
1 ['R1M5NBPRU9URZ1']
0 []
['2015-08-27']
1 ['R286FBMJM6KBVL']
0 []
['2015-08-27']
1 ['RNHNGAHNCNU5W']
0 []
['2015-08-27']
1 ['R1R92GU8HX3MS3']
0 []
['2015-08-27']
1 ['R3R2YD209CFQQB']
0 []
['2015-08-27']
1 ['R1Z4RVBMN5RLYX']
0 []
['2015-08-27']
1 ['R3HFC2B5Y1CF01']


1 ['R243D7ZV85WV1E']
0 []
['2015-08-24']
1 ['R8G3DUEMYGGZF']
0 []
['2015-08-24']
0 []
1 ['R1MT6EWYM9A5RS']
['2015-08-24']
1 ['R1361XYJ9NCLZ3']
0 []
['2015-08-24']
0 []
1 ['RKHDL8NOTEA3B']
['2015-08-24']
1 ['R2QS6V8ZE46VBQ']
0 []
['2015-08-24']
0 []
1 ['R8CUQASNV031M']
['2015-08-24']
1 ['R1ESEZXMRNVIWB']
0 []
['2015-08-24']
1 ['R2PTLOBMINWYI8']
0 []
['2015-08-24']
1 ['R253PAXL7LR1GZ']
0 []
['2015-08-24']
1 ['R2RF6MU1Q0LLSC']
0 []
['2015-08-24']
1 ['RXYQZTNV79JZO']
0 []
['2015-08-24']
0 []
1 ['RWC8SDJU82YT6']
['2015-08-24']
1 ['R3AGL336P2QK6P']
0 []
['2015-08-24']
1 ['R1PZ5OO46UXE7P']
0 []
['2015-08-24']
1 ['R1F2X31IA2YBFT']
0 []
['2015-08-24']
1 ['RM1P5CBRCW1J0']
0 []
['2015-08-24']
1 ['R31SIMB3OF2UTC']
0 []
['2015-08-24']
1 ['R2BM2YEP0S2A2D']
0 []
['2015-08-24']
1 ['R35PXIOJ1Q75VQ']
0 []
['2015-08-24']
1 ['R3JFU3XU7R1CVC']
0 []
['2015-08-24']
1 ['R1E8YDQ6S8VBC0']
0 []
['2015-08-24']
1 ['RXK145HZIBDDY']
0 []
['2015-08-24']
1 ['R1R0U06CD4A3MF']
0 []
['2015-08-24']
0 []
1 ['R2M5F3I1UG7PO0

1 ['R1UYNDRNJH0XWL']
0 []
['2015-08-21']
1 ['RY61AUHFN8ADI']
0 []
['2015-08-21']
1 ['R1SQ40LH3S9VAL']
0 []
['2015-08-21']
0 []
1 ['R4D6TXPTBTYMU']
['2015-08-21']
1 ['RM1L2OI9L36I3']
0 []
['2015-08-21']
1 ['R1R4WMZZTUVW3Q']
0 []
['2015-08-21']
1 ['R1IILT5R4ZD8YA']
0 []
['2015-08-21']
0 []
1 ['R3G00PBY5CEKSZ']
['2015-08-21']
1 ['RQ3LS1ZNRLMA']
0 []
['2015-08-21']
0 []
1 ['R3TOU5NTTR8KJE']
['2015-08-21']
1 ['R37HV0M8VT8VAC']
0 []
['2015-08-21']
1 ['R1TPQNI59ZCZE6']
0 []
['2015-08-21']
1 ['R1I0DHJ0CRDA8W']
0 []
['2015-08-21']
1 ['R1XR716C9XS64']
0 []
['2015-08-21']
1 ['R7TRWT3D0NWVA']
0 []
['2015-08-21']
1 ['R34C2S3GJBJDGG']
0 []
['2015-08-21']
1 ['R115ACF5NH8VCT']
0 []
['2015-08-20']
1 ['R2CNZSHYDNZLF']
0 []
['2015-08-20']
1 ['R2B1J0DV4Q90C']
0 []
['2015-08-20']
1 ['RTV7V99DW7LAR']
0 []
['2015-08-20']
1 ['R3QUX6UDEI7VEJ']
0 []
['2015-08-20']
0 []
1 ['R2OHETRELQHI3L']
['2015-08-20']
1 ['R1MU62FFMJ4URR']
0 []
['2015-08-20']
1 ['R20EQ2PZ10UC5Q']
0 []
['2015-08-20']
1 ['R1MUN0TZJC9MSA']
0 []


KeyError: 749

In [ ]:
## 6th pending

## 7. Removing reviews with no. of verbs > no. of nouns

In [4]:
dataset = pd.read_csv("reviews2.csv",sep='\t')
dataset.drop(axis=0,columns = "Unnamed: 0",inplace=True)
dataset.head(2)

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address,timestamp
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,207.156.210.106,1.441005e+09
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,193.93.167.87,1.441038e+09


In [6]:
fake_review = []
for i in range(len(dataset)):
    #iterate the whole dataset
    
    words = nltk.word_tokenize(dataset["review_body"][i])
    #storing the words from the reviews into the list
    
    tagged_words = nltk.pos_tag(words)
    # returns list of tuples of words along with their parts of speech
    
    nouns_count = 0
    verbs_count = 0
    
    for j in range(len(tagged_words)):
        #iterate through all the words

        if(tagged_words[j][1].startswith("NN")):
            nouns_count+=1
            #counts the no. of nouns in the review

        if(tagged_words[j][1].startswith("VB")):
            verbs_count+=1
            #counts the no. of verbs in the review

    if(verbs_count>nouns_count):
        #comparing the no. of verbs and nouns
        fake_review.append(dataset["review_id"][i])
        #storing the 
        
fake_review

['R8MEA6IGAHO0B',
 'R31LOQ8JGLPRLK',
 'RRB9C05HDOD4O',
 'R26I2RI1GFV8QG',
 'R2AT2426ZHFUHH',
 'RLIL6S9OGM3YA',
 'RT12RQFU0V2AX',
 'R1IG9KI4EWNE4W',
 'R260G1C72KG6KC',
 'R31SFE3UEUA0EY',
 'R174DGBVW0UWAJ',
 'R2074W1A28UUU6',
 'R3B3E2E0123J3F',
 'R3VUB98IS1VKQC',
 'R18TG2USOSVOC3',
 'R2DWIADFBQ3IYG',
 'R2VMBHZ9XTC0GQ',
 'R27GI77VF2P6WF',
 'R35NN211X0RGH9',
 'R18MK5UOS41XJN',
 'R2K2WK38XR5FKZ',
 'R3FBUNWANUJRTU',
 'RGAH8RF1K8QUI',
 'R3KOQ03SYO1FM2',
 'R2TMCO418IRILU',
 'R2H1V2O1SVWULZ']

## 8. Removing reviews with includes more first person pronouns.

In [7]:
fake_reviews = []
for i in range(len(dataset)):
    
    dataset["review_body"][i] = dataset["review_body"][i].lower()
    
    words = nltk.word_tokenize(dataset["review_body"][i])
    sentence = nltk.sent_tokenize(dataset["review_body"][i])
    
    count=0
    
    if(len(sentence)>4):
        # Checking only those reviews which have atleast 5 sentences.
        
        for j in range(len(words)):
            if(words[j]=="i" or words[j]=="we"):
                #counting personal pronouns
                count+=1
                
        if(count > len(sentence)/2):
            #reviews with no of personal pronouns used is greater than half the no. of sentences.
            fake_reviews.append(dataset["customer_id"][i])
fake_reviews

/home/anubhav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


[48950322, 133451, 10757416, 26081688]